<a href="https://colab.research.google.com/github/muhdlaziem/DR/blob/master/Testing_5_undersampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [ ]:
%tensorflow_version 1.x

# import libraries
import json
import math
from tqdm import tqdm, tqdm_notebook
import gc
import warnings
import os

import cv2
from PIL import Image

import pandas as pd
import scipy
import matplotlib.pyplot as plt

from keras import backend as K
from keras import layers
from keras.applications.densenet import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import numpy as np

warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# Image size
im_size = 320

In [ ]:
import pandas as pd
# %cd diabetic-retinopathy-resized/
DR = pd.read_csv('/gdrive/My Drive/diabetic-retinopathy-resized/MadamAmeliaSample/label_for_out_MYRRC_data2_256x256.csv')
DR.head()

,file,class
0,00000.jpg,0
1,00001.jpg,0
2,00002.jpg,0
3,00003.jpg,0
4,00004.jpg,0


In [ ]:
def preprocess_image(image_path, desired_size=224):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = crop_image_from_gray(img)
    img = cv2.resize(img, (desired_size,desired_size))
    img = cv2.addWeighted(img,4,cv2.GaussianBlur(img, (0,0), desired_size/40) ,-4 ,128)
    
    return img

In [ ]:
# testing set
%cd /gdrive/My Drive/diabetic-retinopathy-resized/MadamAmeliaSample/image
N = DR.shape[0]
x_test = np.empty((N, im_size, im_size, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm_notebook(DR['file'])):
    x_test[i, :, :, :] = preprocess_image(
        f'{image_id}',
        desired_size = im_size
    )

/gdrive/My Drive/diabetic-retinopathy-resized/MadamAmeliaSample/image


In [ ]:
y_test = pd.get_dummies(DR['class']).values

print(y_test.shape)
print(x_test.shape)

(60, 3)
(60, 320, 320, 3)


In [ ]:
y_test_multi = np.empty(y_test.shape, dtype=y_test.dtype)
y_test_multi[:, 2] = y_test[:, 2]

for i in range(1, -1, -1):
    y_test_multi[:, i] = np.logical_or(y_test[:, i], y_test_multi[:, i+1])

print("Y_test multi: {}".format(y_test_multi.shape))

Y_test multi: (60, 3)


In [ ]:
# from keras.models import load_model
import tensorflow as tf

model = tf.keras.models.load_model('/gdrive/My Drive/diabetic-retinopathy-resized/resized_train_cropped/denseNet_5.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 10, 10, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 7,042,629
Trainable params: 6,958,981
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
y_val_pred = model.predict(x_test)                                                                        

In [ ]:
def compute_score_inv(threshold):
    y1 = y_val_pred > threshold
    y1 = y1.astype(int).sum(axis=1) - 1
    y2 = y_test_multi.sum(axis=1) - 1
    score = cohen_kappa_score(y1, y2, weights='quadratic')
    
    return 1 - score

simplex = scipy.optimize.minimize(
    compute_score_inv, 0.5, method='nelder-mead'
)

best_threshold = simplex['x'][0]

y1 = y_val_pred > best_threshold
y1 = y1.astype(int).sum(axis=1) - 1
y1 = np.where(y1==2,1,y1)
y1 = np.where(y1==3,2,y1)
y1 = np.where(y1==4,2,y1)

y2 = y_test_multi.sum(axis=1) - 1
score = cohen_kappa_score(y1, y2, weights='quadratic')
print('Threshold: {}'.format(best_threshold))
print('Validation QWK score with best_threshold: {}'.format(score))

y1 = y_val_pred > .5
y1 = y1.astype(int).sum(axis=1) - 1
y1 = np.where(y1==2,1,y1)
y1 = np.where(y1==3,2,y1)
y1 = np.where(y1==4,2,y1)
score = cohen_kappa_score(y1, y2, weights='quadratic')
print('Validation QWK score with .5 threshold: {}'.format(score))

Threshold: 0.525
Validation QWK score with best_threshold: 0.49275362318840576
Validation QWK score with .5 threshold: 0.4571428571428572


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_best = y_val_pred > best_threshold
y_best = y_best.astype(int).sum(axis=1) - 1
y_best = np.where(y_best==2,1,y_best)
y_best = np.where(y_best==3,2,y_best)
y_best = np.where(y_best==4,2,y_best)

print('Confusion Matrix')
print(confusion_matrix(y_best, y2))
print('Classification Report')
target_names = ['No DR', 'Moderate', 'Severe']
print(classification_report(y_best, y2, target_names=target_names))

Confusion Matrix
[[ 1  0  0]
 [16 14  1]
 [ 3  6 19]]
Classification Report
              precision    recall  f1-score   support

       No DR       0.05      1.00      0.10         1
    Moderate       0.70      0.45      0.55        31
      Severe       0.95      0.68      0.79        28

    accuracy                           0.57        60
   macro avg       0.57      0.71      0.48        60
weighted avg       0.81      0.57      0.65        60



In [ ]:
print(y_best)
print(y2)

[1 0 1 1 2 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1
 2 1 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
DR['predicted'] = y_best
DR.to_excel("/gdrive/My Drive/diabetic-retinopathy-resized/MadamAmeliaSample/result_5_undersampling.xlsx")